# Data step - przetwarzanie danych

Do przetwarzania danych w SAS wykorzystuje się *data stepy*:

```
data nazwa_nowego_zbioru;
set nazwa_bazowego_zbioru;
<modyfikacje>;
run;
```

Argumenty, które można podać w data stepie to m.in.:

- `keep` - wybór kolumn, które mają pozostać w nowym zbiorze,
- `drop` - wybór kolumn, które mają zostać pominięte w nowym zbiorze,
- `where` - filtrowanie obserwacji,
- `rename` - zmiana nazwy zmiennej (`stara_nazwa=nowa_nazwa`),
- `label` - nadanie etykiety zmiennej.

Argumenty można podawać w ciele data stepu, np:

```
data bank2;
set bank;
where wiek > 50;
run;
```

Wówczas nie jest ważna kolejność podawania argumentów i korzysta się z oryginalnych nazw zmiennych.

Argumenty można także deklarować jako opcję zbioru, np:

```
data bank2;
set bank(where = (wiek > 50));
run;
```

W takim przypadku instrukcje są wykonywane alfabetycznie, zatem trzeba zwracać uwagę na wykonywane modyfikacje.

In [1]:
libname bib "/folders/myfolders/przetwarzanie-danych-SAS/";

SAS Connection established. Subprocess id is 3233



W pierwszej kolejności przeanalizujemy wyniki działania argumentów `keep` i `drop`:

In [4]:
data wiek;
set bib.bank;
keep wiek;
run;

proc print data=wiek(obs=5);
run;

Obs,wiek
1,58
2,44
3,33
4,47
5,33


In [5]:
data no_wiek;
set bib.bank;
drop wiek;
run;

proc print data=no_wiek(obs=5);
run;

Obs,praca,stan_cywil,wykszt,kredyt,saldo,hipoteka,pozyczka,kontakt,dzien,miesiac,czas,licz_kontakt,dni_kontakt,licz_kontakt_poprz,wynik_poprz,wynik
1,3,2,wyzsze,nie,2143,tak,nie,.,5,5,261,1,-1,0,.,nie
2,9,1,srednie,nie,29,tak,nie,.,5,5,151,1,-1,0,.,nie
3,5,2,srednie,nie,2,tak,tak,.,5,5,76,1,-1,0,.,nie
4,6,2,,nie,1506,tak,nie,.,5,5,92,1,-1,0,.,nie
5,.,1,,nie,1,nie,nie,.,5,5,198,1,-1,0,.,nie


Do filtrowania można wykorzystać bardzo wiele operatorów porównań:

| Symbol | Ekwiwalent | Definicja |
|------|----------|---------|
| = | eq | równe|
| ^= / ~= / <> | ne | różne |
| > | gt | większe |
| < | lt | mniejsze |
| >= | ge | większe lub równe |
| <= | le | mniejsze lub równe |
| between ... and ... | | w przedziale |
| contains | | zawiera |

In [21]:
data saldo_ge_1000;
set bib.bank;
where saldo >= 1000 and wynik="tak";
run;

proc means data=saldo_ge_1000;
var saldo;
run;

proc freq data=saldo_ge_1000;
table wynik;
run;

Przy zmianie nazwy zmiennej najpierw podajemy starą nazwę zmiennej, a po znaku równości nową.

In [13]:
data bank2;
set bib.bank;
rename kredyt=karta;
run;

proc print data=bank2(obs=5);
var karta;
run;

Obs,karta
1,nie
2,nie
3,nie
4,nie
5,nie


Do zmiennej można także przypisać etykietę - będzie się ona pojawiała w wygenerowanych raportach i z racji tego, że będzie ustalona na poziomie zbioru to nie będzie potrzeby deklarowania jej osobno np. w `proc freq`.

In [14]:
data bank2;
set bank2;
label karta="Czy klient posiada kartę kredytową";
run;

proc freq data=bank2;
table karta;
run;

Modyfikacja oraz tworzenie nowej zmiennej w *data step* polega na podaniu nazwy nowej/modyfikowanej zmiennej i po znaku równości wartości jakie ma przyjmować.

In [15]:
data bank2;
set bank2;
saldo_prom=saldo+500;
run;

proc print data=bank2(obs=5);
var saldo saldo_prom;
run;

Obs,saldo,saldo_prom
1,2143,2643
2,29,529
3,2,502
4,1506,2006
5,1,501


W *data stepie* można także korzystać z warunkowego przetwarzania:

In [17]:
data bank2;
set bank2;
if wynik="tak" and saldo > 0 then saldo_wynik=1.05*saldo;
else saldo_wynik=saldo;
run;

proc means data=bank2 min mean max;
var saldo saldo_wynik;
class wynik;
run;

wynik,N Obs,Variable,Label,N,Mean,Std Dev,Minimum,Maximum
nie,39922,saldo saldo_wynik,saldo,39922 39922,1303.71 1303.71,2974.20 2974.20,-8019.00 -8019.00,102127.00 102127.00
tak,5289,saldo saldo_wynik,saldo,5289 5289,1804.27 1895.08,3501.10 3675.74,-3058.00 -3058.00,81204.00 85264.20


Ten sam mechanizm można także wykorzystać do rekodowania zmiennej (niemniej dużo lepiej skorzystać z formatów):

In [23]:
data bank2;
set bank2;
if wiek < 20 then 
gwiek=1;
else if wiek >= 20 and wiek < 40 then
gwiek=2;
else if wiek ge 40 and wiek lt 60 then
gwiek=3;
else 
gwiek=4;
run;


proc means data=bank2 min max;
var wiek;
class gwiek;
run;

Proszę utworzyć zbiór o nazwie `saldo_60` na podstawie zbioru bank zawierający zmienne wiek, praca, saldo, przy czym mają to być tylko osoby powyżej 60 roku życia, a zmienna praca ma mieć zmienioną nazwę na zawod, natomiast zmienna wiek ma mieć nadaną etykietę Wiek klienta. Ile wynosi średni wiek osób z tego zbioru?